## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-20-16-52-09 +0000,nyt,Trump Says He’s Ended 6 (or 7) Wars. Here’s So...,https://www.nytimes.com/2025/08/19/world/europ...
1,2025-08-20-16-50-15 +0000,nyt,Judge Halts Texas’ Law Mandating the Ten Comma...,https://www.nytimes.com/2025/08/20/us/politics...
2,2025-08-20-16-50-13 +0000,nyt,N.S.A.’s Acting Director Tried to Save Top Sci...,https://www.nytimes.com/2025/08/20/us/politics...
3,2025-08-20-16-47-49 +0000,nypost,Mayor Adams’ longtime confidant Ingrid Lewis-M...,https://nypost.com/2025/08/20/us-news/mayor-ad...
4,2025-08-20-16-46-52 +0000,bbc,German controversy surrounds jail term for tra...,https://www.bbc.com/news/articles/c3r4zrg35vlo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,42
576,ukraine,16
130,new,13
1,he,12
409,up,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
235,2025-08-19-22-54-43 +0000,nypost,Russia launches largest strike on Ukraine in w...,https://nypost.com/2025/08/19/world-news/russi...,103
280,2025-08-19-20-30-23 +0000,latimes,Contributor: Trump's Russia and Ukraine summit...,https://www.latimes.com/opinion/story/2025-08-...,101
77,2025-08-20-13-19-11 +0000,nypost,At least 14 wounded in Russian attack on Ukrai...,https://nypost.com/2025/08/20/world-news/russi...,85
57,2025-08-20-14-24-11 +0000,bbc,Kremlin plays down Zelensky talks as Trump war...,https://www.bbc.com/news/articles/cn92e52rpjxo...,82
38,2025-08-20-15-35-01 +0000,nypost,White House trolls Mark Hamill over claim he n...,https://nypost.com/2025/08/20/us-news/white-ho...,81


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
235,103,2025-08-19-22-54-43 +0000,nypost,Russia launches largest strike on Ukraine in w...,https://nypost.com/2025/08/19/world-news/russi...
19,56,2025-08-20-16-20-48 +0000,nypost,Mark Zuckerberg’s pricey new AI hires — includ...,https://nypost.com/2025/08/20/business/mark-zu...
332,48,2025-08-19-18-16-27 +0000,nyt,Why Were Air Canada Flight Attendants Striking...,https://www.nytimes.com/2025/08/19/business/ai...
292,48,2025-08-19-19-55-47 +0000,nypost,Texas Dem Rep. Nicole Collier slept in state H...,https://nypost.com/2025/08/19/us-news/texas-de...
186,47,2025-08-20-03-12-54 +0000,nyt,He Tried to Endorse From the Pulpit. He Wound ...,https://www.nytimes.com/2025/08/19/us/politics...
18,38,2025-08-20-16-21-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/politics/policy/fed-lisa-c...
267,38,2025-08-19-21-04-20 +0000,nypost,Pro-Adams PAC calling NYC voters to rip ‘sore ...,https://nypost.com/2025/08/19/us-news/pro-adam...
200,37,2025-08-20-01-29-42 +0000,cbc,Kristi Noem says U.S. will paint Mexico border...,https://www.cbc.ca/news/world/noem-us-border-w...
120,36,2025-08-20-10-00-00 +0000,nypost,Here’s the status of Russia’s meat-grinder off...,https://nypost.com/2025/08/20/world-news/heres...
61,36,2025-08-20-14-13-02 +0000,bbc,South African MP opens fire to fend off attack...,https://www.bbc.com/news/articles/cm2k167kge2o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
